In [ ]:
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
% cd /content/gdrive/My\ Drive/Colab\ Notebooks/nlp_task/bert_cla
! pwd
! ls

/content/gdrive/My Drive/Colab Notebooks/nlp_task/bert_cla
/content/gdrive/My Drive/Colab Notebooks/nlp_task/bert_cla
albert_large	global_config.py	run_multi_label_linear.py
albert_tiny	__init__.py		runs
apex		main.py			train.tsv
base_runner.py	models			val.tsv
base_utils.py	outputs			wandb
cache_dir	outputs_tiny		xbert_cnn_runner.py
cooking.train	__pycache__		xbert_multi_label_linear_runner.py
cooking.valid	run_albert_large.ipynb	xbert_rnn_runner.py
data		run_albert_tiny.ipynb
fasttext.ipynb	run_multi_class_cnn.py


In [ ]:
# !pip install wandb tensorboardX transformers
!pip install fasttext jieba

     |████████████████████████████████| 71kB 2.2MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3017359 sha256=c1655fba991328f35cbbadb3fe0f14d0524761656ea2c1b64fe71ec105c85498
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [2]:
import fasttext
import re  #正则表达式
import pandas as pd
import jieba
import tqdm

In [3]:
train = pd.read_table('train.tsv', names=['label', 'content'], encoding='utf-8')
valid = pd.read_table('val.tsv', names=['label', 'content'], encoding='utf-8')

In [ ]:
import time

s_time = time.time()
train['content_cut'] = train.content.apply(lambda x:  ' '.join(jieba.cut(x)))
valid['content_cut'] = valid.content.apply(lambda x:  ' '.join(jieba.cut(x)))
print("time: ", time.time() - s_time)

In [ ]:
train.head(5)

,label,content,content_cut
0,0_-2@1_-2@2_-2@3_-2@4_1@5_-2@6_-2@7_-2@8_-2@9_...,吼吼吼～萌死人的棒棒糖～中了大众点评的霸王餐～太可爱了。一直就好奇这个棒棒糖是怎么个东西～大...,吼吼 吼 ～ 萌死 人 的 棒棒糖 ～ 中 了 大众 点评 的 霸王餐 ～ 太 可爱 了 。...
1,0_-2@1_-2@2_-2@3_-2@4_-2@5_-2@6_-2@7_0@8_-2@9_...,第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等～其次霸王餐提供...,第三次 参加 大众 点评 网 霸王餐 的 活动 。 这家 店 给 人 整体 感觉 一般 。 ...
2,0_-2@1_-2@2_-2@3_-2@4_0@5_-2@6_1@7_0@8_-2@9_1@...,4人同行点了10个小吃榴莲酥榴莲味道不足松软奶味浓虾饺好吃两颗大虾仁皮蛋粥皮蛋多但是一般挺稠...,4 人 同行 点 了 10 个 小吃 榴莲 酥 榴莲 味道 不足 松软 奶味 浓 虾饺 好吃...
3,0_-2@1_-2@2_-2@3_-2@4_-2@5_-2@6_-2@7_0@8_-2@9_...,之前评价了莫名其妙被删果断继续差评！换了菜单价格更低开始砸牌子但套餐还是有150的点了两份套...,之前 评价 了 莫名其妙 被删 果断 继续 差评 ！ 换 了 菜单 价格 更 低 开始 砸牌...
4,0_-2@1_-2@2_-2@3_-2@4_-2@5_-2@6_-2@7_-2@8_-2@9...,出乎意料地惊艳～椰子鸡清热降火～美容养颜～大大满足了爱吃火锅怕上火星人。椰子冻是帅帅的老板原...,出乎意料 地 惊艳 ～ 椰子 鸡 清热 降火 ～ 美容 养颜 ～ 大大 满足 了 爱 吃火锅...


In [5]:
def label_string(label):
    labs = ''
    for lab in label.split('@'):
        labs += "\t__label__" + str(lab)
    return labs


def single_label_string(label, idx):
    "idx: 0 - 19"
    return "\t__label__" + str(label.split('@')[idx])


def single_class_string(label, idx, cla):
    "idx: 0 - 19, cla: [-2 -1 0 1]"
    lab = f"{idx}_{cla}"
    if str(label.split('@')[idx]) == lab:
        out = "\t__label__" + "1"
    else:
        out = "\t__label__" + "0"
    return out

In [6]:
def to_file(file, x, y, label_func):
    for i, line in tqdm.tqdm_notebook(enumerate(x)):
        outline =  line + label_func(y[i]) + "\n"
        file.write(outline)
        if i % 50000 == 0:
            print(outline)
    file.close()

In [11]:
def gen_train_file_all():
    "全label联合输入"
    ftrain = open(f"data/sentiment_train.txt", "w", encoding='utf-8')
    ftest = open(f"data/sentiment_test.txt", "w", encoding='utf-8')

    to_file(ftrain, train.content_cut, train.label, label_string)
    to_file(ftest, valid.content_cut, valid.label, label_string)


def gen_train_file_label(label_idx):
    "每一类label softmax"
    ftrain = open(f"data/sentiment_train_{label_idx}.txt", "w", encoding='utf-8')
    ftest = open(f"data/sentiment_test_{label_idx}.txt", "w", encoding='utf-8')

    to_file(ftrain, train.content, train.label, lambda label: single_label_string(label, label_idx))
    to_file(ftest, valid.content, valid.label, lambda label: single_label_string(label, label_idx))


def gen_train_file_cla(label_idx, cla):
    "每一类label的每一个class进行 0 1 二分类"
    ftrain = open(f"data/sentiment_train_{label_idx}_{cla}.txt", "w", encoding='utf-8')
    ftest = open(f"data/sentiment_test_{label_idx}_{cla}.txt", "w", encoding='utf-8')

    to_file(ftrain, train.content_cut, train.label, lambda label: single_class_string(label, label_idx, cla))
    to_file(ftest, valid.content_cut, valid.label, lambda label: single_class_string(label, label_idx, cla))

In [9]:
from fasttext import train_supervised


def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@ \t{:.3f}".format(p))
    print("R@ \t{:.3f}".format(r))
    print("f1@ \t {:.3f}".format(2 * p * r / (p + r)))


def train_all():
    model = train_supervised(
        input="data/sentiment_train.txt",
        epoch=30,
        lr=0.5,
        dim=100,
        loss='softmax',
        wordNgrams=2,
        verbose=1,
        minCount=1)
    return model

def test_model_all(model):
    print_results(*model.test(f"data/sentiment_test.txt"))


def train_label(idx):
    model = train_supervised(
        input=f"data/sentiment_train_{idx}.txt",
        epoch=30,
        lr=1.0,
        dim=100,
        loss='hs',
        wordNgrams=2,
        verbose=1,
        minCount=1)
    return model

def test_model_label(model, idx):
    print_results(*model.test(f"data/sentiment_test_{idx}.txt"))


def train_cla(idx, cla):
    model = train_supervised(
            input=f"data/sentiment_train_{idx}_{cla}.txt",
            epoch=30,
            lr=0.8,
            dim=100,
            loss='ns',
            wordNgrams=2,
            verbose=1,
            minCount=1)
    return model

def test_model_cla(model, idx, cla):
    print_results(*model.test(f"data/sentiment_test_{idx}_{cla}.txt"))

In [ ]:
# gen_train_file_cla(1, -2)
# model_1_n2 = train_cla(1, -2)
# test_model_cla(model_1_n2, 1, -2)

In [ ]:
# gen_train_file_label(1)
# model_1 = train_label(1)
# test_model_label(model_1, 1)

In [ ]:
def main_train_label():
    for idx in range(20):
        start_time = time.time()
        print("=" * 30)
        gen_train_file_label(idx)
        print("File generated! ")

        model = train_label(idx)
        print("Train finished! ")

        print(f"Label {idx}: ")
        test_model_label(model, idx)

        model.save_model(f"data/model_{idx}")
        print("Model saved! ")

        print("Time spent: ", time.time() - start_time)

        del model

In [ ]:
# main_train_label()

### 超参数搜索

In [10]:
# hyperparameter optimization

def search_label(duration=600):
    model = train_supervised(input=f"data/sentiment_train.txt", 
                                             autotuneValidationFile=f"data/sentiment_test.txt",
                                             autotuneMetric="f1",
                                             autotuneDuration=duration,
                                             verbose=0)
    return model


def search_label(idx, duration=600):
    model = train_supervised(input=f"data/sentiment_train_{idx}.txt", 
                                             autotuneValidationFile=f"data/sentiment_test_{idx}.txt",
                                             autotuneMetric="f1",
                                             autotuneDuration=duration,
                                             verbose=0)
    return model


def search_cla(idx, cla, duration=600):
    model = train_supervised(input=f"data/sentiment_train_{idx}_{cla}.txt", 
                                             autotuneValidationFile=f"data/sentiment_test_{idx}_{cla}.txt",
                                             autotuneMetric="f1",
                                             autotuneDuration=duration,
                                             verbose=0)
    return model

In [ ]:
# model_opt_1 = search_label(1)

In [ ]:
# test_model_label(model_opt_1, 1)

N	15000
P@ 	0.863
R@ 	0.863
f1@ 	 0.863


In [ ]:
# model_opt_1.save_model('test')

In [ ]:
def main_train_label():
    for idx in range(20):
        start_time = time.time()

        print("=" * 30)
        gen_train_file_label(idx)
        print("File generated! ")

        model_opt = search_label(idx, duration=600)
        print("Search finished! ")

        print(f"Label {idx}: ")
        test_model_label(model_opt, idx)

        model_opt.save_model(f"data/model_opt_{idx}")
        print("Model saved! ")

        print("Time spent: ", time.time() - start_time)

        del model_opt

In [ ]:
main_train_label()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


吼吼 吼 ～ 萌死 人 的 棒棒糖 ～ 中 了 大众 点评 的 霸王餐 ～ 太 可爱 了 。 一直 就 好奇 这个 棒棒糖 是 怎么 个 东西 ～ 大众 点评 给 了 我 这个 土老冒 一个 见识 的 机会 。 看 介绍 棒棒糖 是 用 德国 糖 做 的 ～ 不会 很甜 ～ 中间 的 照片 是 糯米 的 ～ 能 食用 ～ 真是太 高端 大气 上档次 了 ～ 还 可以 买 蝴蝶结 扎口 ～ 送人 可以 买 礼盒 。 我 是 先 打 的 卖家 电话 ～ 加 了 微信 ～ 给 卖家 传 的 照片 。 等 了 几天 ～ 卖家 就 告诉 我 可以 取货 了 ～ 去 大官 屯 那取 的 。 虽然 连 卖家 的 面 都 没 见到 ～ 但是 还是 谢谢 卖家 送 我 这么 可爱 的 东西 ～ 太 喜欢 了 ～ 这 哪 舍得吃 啊 。	__label__0_-2

餐前 ～ 装 冷菜 的 碟子 是 仿 青瓷 的 碟子 ～ 高脚 ～ 可惜 是 塑料 的 ～ 近 看 缺 了 质感 ～ 没有 瓷器 那种 通透 ～ 远看 挺不错 。 盛菜 的 器皿 有些 也 挺 有 特色 的 ～ 包厢 布置 也 花 了 功夫 ～ 走 古风 挂 字画 。 墙纸 是 青花瓷 的 ～ 对面 墙壁 也 贴 了 很多 瓷器 ～ 有 几个 还是 玲珑 瓷 ～ 透光 看 比较 好看 。 比较 喜欢 他们 家 的 日式 茶壶 ～ 铁壶 ～ 很沉 但 漂亮 ～ 据说 冲泡 能 补铁 ？ 菜品 味道 一般 ～ 花样 是 挺 多 的 ～ 杨枝 甘露 换个 器皿 就 能 换来 很大 的 拍照 量 ～ 是不是 桌桌 都 有 ？ 喜欢 那个 有 小 把 的 酱料 器皿 ～ 拿 着 很 可爱	__label__0_-2

非常 喜欢 去 新开 的 有 特色 的 店 品尝 ～ 有幸 被 大众 点评 网同城 聚会 抽到 黄金 替补 免费 品鉴 的 机会 ～ 我 感到 十分 幸运 ！ 红 不让 台湾 特色 餐厅 位于 李沧 万达 金街 1 号 入口 扶梯 旁 ～ 交通 便利 ～ 停车 方便 。 店面 小资 ～ 适合 举行 小型 派对 。 走近 店门 ～ 服务员 立刻 开门 迎接 并且 热情 的 打招呼 。 进入 店面 ～ 环顾四周 ～ 鹅黄 的 灯光 下 透露 出 温馨 慵懒 的 气氛 ～ 店家 匠心 独居 ～ 屋内 的 摆设 典雅 质朴 。

哎 ～ 想当年 来 佘山 的 时候 ～ 啥 都 没有 ～ 三品 香算 镇上 最大 看起来 最 像样 的 饭店 了 。 菜品 多 ～ 有点 太 多 ～ 感觉 啥 都 有 ～ 杂都 不足以 形容 。 随便 点些 ～ 居然 口味 什么 的 都 好 还 可以 ～ 价钱 自然 是 便宜 当 震惊 。 元宝 虾 和 椒盐 九肚鱼 都 不错 吃 。 不过 近来 几次 么 ～ 味道 明显 没 以前 好 了 。 冷餐 里面 一个 凉拌 海带丝 还 可以 ～ 酸酸甜甜 的 。 镇上 也 有 了 些 别的 大点 的 饭店 ～ 所以 不是 每次 必来 了 。 对 了 ～ 这家 的 生意 一如既往 的 超级 好 ～ 不 定位 基本 吃 不到 。 不过 佘山 这边 的 人 吃晚饭 很早 的 ～ 所以 稍微 晚点 去 就 很 空 了 。	__label__0_-2


File generated! 
Search finished! 
Label 0: 
N	15000
P@ 	0.924
R@ 	0.924
f1@ 	 0.924
Model saved! 
Time spent:  864.0277895927429


吼吼 吼 ～ 萌死 人 的 棒棒糖 ～ 中 了 大众 点评 的 霸王餐 ～ 太 可爱 了 。 一直 就 好奇 这个 棒棒糖 是 怎么 个 东西 ～ 大众 点评 给 了 我 这个 土老冒 一个 见识 的 机会 。 看 介绍 棒棒糖 是 用 德国 糖 做 的 ～ 不会 很甜 ～ 中间 的 照片 是 糯米 的 ～ 能 食用 ～ 真是太 高端 大气 上档次 了 ～ 还 可以 买 蝴蝶结 扎口 ～ 送人 可以 买 礼盒 。 我 是 先 打 的 卖家 电话 ～ 加 了 微信 ～ 给 卖家 传 的 照片 。 等 了 几天 ～ 卖家 就 告诉 我 可以 取货 了 ～ 去 大官 屯 那取 的 。 虽然 连 卖家 的 面 都 没 见到 ～ 但是 还是 谢谢 卖家 送 我 这么 可爱 的 东西 ～ 太 喜欢 了 ～ 这 哪 舍得吃 啊 。	__label__1_-2

餐前 ～ 装 冷菜 的 碟子 是 仿 青瓷 的 碟子 ～ 高脚 ～ 可惜 是 塑料 的 ～ 近 看 缺 了 质感 ～ 没有 瓷器 那种 通透 ～ 远看 挺不错 。 盛菜 的 器皿 有些 也 挺 有 特色 的 ～ 包厢 布置 也 花 了 功夫 ～ 走 古风 挂 字画 。 墙纸 是 青花瓷 的 ～ 对面 墙壁 也 贴 了 很多 瓷器 ～ 有 几个 还是 玲珑 瓷 ～ 透光 看 比较 好看 。 比较 喜欢 他们 家 的 日式 茶壶 ～ 铁壶 ～ 很沉 但 漂亮 ～ 据说 冲泡 能 补铁 ？ 菜品 味道 一般 ～ 花样 是 挺 多 的 ～ 杨枝 甘露 换个 器皿 就 能 换来 很大 的 拍照 量 ～ 是不是 桌桌 都 有 ？ 喜欢 那个 有 小 把 的 酱料 器皿 ～ 拿 着 很 可爱	__label__1_-2

非常 喜欢 去 新开 的 有 特色 的 店 品尝 ～ 有幸 被 大众 点评 网同城 聚会 抽到 黄金 替补 免费 品鉴 的 机会 ～ 我 感到 十分 幸运 ！ 红 不让 台湾 特色 餐厅 位于 李沧 万达 金街 1 号 入口 扶梯 旁 ～ 交通 便利 ～ 停车 方便 。 店面 小资 ～ 适合 举行 小型 派对 。 走近 店门 ～ 服务员 立刻 开门 迎接 并且 热情 的 打招呼 。 进入 店面 ～ 环顾四周 ～ 鹅黄 的 灯光 下 透露 出 温馨 慵懒 的 气氛 ～ 店家 匠心 独居 ～ 屋内 的 摆设 典雅 质朴 。

哎 ～ 想当年 来 佘山 的 时候 ～ 啥 都 没有 ～ 三品 香算 镇上 最大 看起来 最 像样 的 饭店 了 。 菜品 多 ～ 有点 太 多 ～ 感觉 啥 都 有 ～ 杂都 不足以 形容 。 随便 点些 ～ 居然 口味 什么 的 都 好 还 可以 ～ 价钱 自然 是 便宜 当 震惊 。 元宝 虾 和 椒盐 九肚鱼 都 不错 吃 。 不过 近来 几次 么 ～ 味道 明显 没 以前 好 了 。 冷餐 里面 一个 凉拌 海带丝 还 可以 ～ 酸酸甜甜 的 。 镇上 也 有 了 些 别的 大点 的 饭店 ～ 所以 不是 每次 必来 了 。 对 了 ～ 这家 的 生意 一如既往 的 超级 好 ～ 不 定位 基本 吃 不到 。 不过 佘山 这边 的 人 吃晚饭 很早 的 ～ 所以 稍微 晚点 去 就 很 空 了 。	__label__1_-2


File generated! 
Search finished! 
Label 1: 
N	15000
P@ 	0.863
R@ 	0.863
f1@ 	 0.863
Model saved! 
Time spent:  860.4216887950897


...

哎 ～ 想当年 来 佘山 的 时候 ～ 啥 都 没有 ～ 三品 香算 镇上 最大 看起来 最 像样 的 饭店 了 。 菜品 多 ～ 有点 太 多 ～ 感觉 啥 都 有 ～ 杂都 不足以 形容 。 随便 点些 ～ 居然 口味 什么 的 都 好 还 可以 ～ 价钱 自然 是 便宜 当 震惊 。 元宝 虾 和 椒盐 九肚鱼 都 不错 吃 。 不过 近来 几次 么 ～ 味道 明显 没 以前 好 了 。 冷餐 里面 一个 凉拌 海带丝 还 可以 ～ 酸酸甜甜 的 。 镇上 也 有 了 些 别的 大点 的 饭店 ～ 所以 不是 每次 必来 了 。 对 了 ～ 这家 的 生意 一如既往 的 超级 好 ～ 不 定位 基本 吃 不到 。 不过 佘山 这边 的 人 吃晚饭 很早 的 ～ 所以 稍微 晚点 去 就 很 空 了 。	__label__7_1


File generated! 
Search finished! 
Label 7: 
N	15000
P@ 	0.697
R@ 	0.697
f1@ 	 0.697
Model saved! 
Time spent:  853.069935798645


吼吼 吼 ～ 萌死 人 的 棒棒糖 ～ 中 了 大众 点评 的 霸王餐 ～ 太 可爱 了 。 一直 就 好奇 这个 棒棒糖 是 怎么 个 东西 ～ 大众 点评 给 了 我 这个 土老冒 一个 见识 的 机会 。 看 介绍 棒棒糖 是 用 德国 糖 做 的 ～ 不会 很甜 ～ 中间 的 照片 是 糯米 的 ～ 能 食用 ～ 真是太 高端 大气 上档次 了 ～ 还 可以 买 蝴蝶结 扎口 ～ 送人 可以 买 礼盒 。 我 是 先 打 的 卖家 电话 ～ 加 了 微信 ～ 给 卖家 传 的 照片 。 等 了 几天 ～ 卖家 就 告诉 我 可以 取货 了 ～ 去 大官 屯 那取 的 。 虽然 连 卖家 的 面 都 没 见到 ～ 但是 还是 谢谢 卖家 送 我 这么 可爱 的 东西 ～ 太 喜欢 了 ～ 这 哪 舍得吃 啊 。	__label__8_-2

餐前 ～ 装 冷菜 的 碟子 是 仿 青瓷 的 碟子 ～ 高脚 ～ 可惜 是 塑料 的 ～ 近 看 缺 了 质感 ～ 没有 瓷器 那种 通透 ～ 远看 挺不错 。 盛菜 的 器皿 有些 也 挺 有 特色 的 ～ 包厢 布置 也 花 了 功夫 ～ 走 古风 挂 字画 。 墙纸 是 青花瓷 的 ～ 对面 墙壁 也 贴 了 很多 瓷器 ～ 有 几个 还是 玲珑 瓷 ～ 透光 看 比较 好看 。 比较 喜欢 他们 家 的 日式 茶壶 ～ 铁壶 ～ 很沉 但 漂亮 ～ 据说 冲泡 能 补铁 ？ 菜品 味道 一般 ～ 花样 是 挺 多 的 ～ 杨枝 甘露 换个 器皿 就 能 换来 很大 的 拍照 量 ～ 是不是 桌桌 都 有 ？ 喜欢 那个 有 小 把 的 酱料 器皿 ～ 拿 着 很 可爱	__label__8_-2

非常 喜欢 去 新开 的 有 特色 的 店 品尝 ～ 有幸 被 大众 点评 网同城 聚会 抽到 黄金 替补 免费 品鉴 的 机会 ～ 我 感到 十分 幸运 ！ 红 不让 台湾 特色 餐厅 位于 李沧 万达 金街 1 号 入口 扶梯 旁 ～ 交通 便利 ～ 停车 方便 。 店面 小资 ～ 适合 举行 小型 派对 。 走近 店门 ～ 服务员 立刻 开门 迎接 并且 热情 的 打招呼 。 进入 店面 ～ 环顾四周 ～ 鹅黄 的 灯光 下 透露 出 温馨 慵懒 的 气氛 ～ 店家 匠心 独居 ～ 屋内 的 摆设 典雅 质朴 。

哎 ～ 想当年 来 佘山 的 时候 ～ 啥 都 没有 ～ 三品 香算 镇上 最大 看起来 最 像样 的 饭店 了 。 菜品 多 ～ 有点 太 多 ～ 感觉 啥 都 有 ～ 杂都 不足以 形容 。 随便 点些 ～ 居然 口味 什么 的 都 好 还 可以 ～ 价钱 自然 是 便宜 当 震惊 。 元宝 虾 和 椒盐 九肚鱼 都 不错 吃 。 不过 近来 几次 么 ～ 味道 明显 没 以前 好 了 。 冷餐 里面 一个 凉拌 海带丝 还 可以 ～ 酸酸甜甜 的 。 镇上 也 有 了 些 别的 大点 的 饭店 ～ 所以 不是 每次 必来 了 。 对 了 ～ 这家 的 生意 一如既往 的 超级 好 ～ 不 定位 基本 吃 不到 。 不过 佘山 这边 的 人 吃晚饭 很早 的 ～ 所以 稍微 晚点 去 就 很 空 了 。	__label__8_-2


File generated! 
Search finished! 
Label 8: 
N	15000
P@ 	0.858
R@ 	0.858
f1@ 	 0.858
Model saved! 
Time spent:  863.2569625377655


吼吼 吼 ～ 萌死 人 的 棒棒糖 ～ 中 了 大众 点评 的 霸王餐 ～ 太 可爱 了 。 一直 就 好奇 这个 棒棒糖 是 怎么 个 东西 ～ 大众 点评 给 了 我 这个 土老冒 一个 见识 的 机会 。 看 介绍 棒棒糖 是 用 德国 糖 做 的 ～ 不会 很甜 ～ 中间 的 照片 是 糯米 的 ～ 能 食用 ～ 真是太 高端 大气 上档次 了 ～ 还 可以 买 蝴蝶结 扎口 ～ 送人 可以 买 礼盒 。 我 是 先 打 的 卖家 电话 ～ 加 了 微信 ～ 给 卖家 传 的 照片 。 等 了 几天 ～ 卖家 就 告诉 我 可以 取货 了 ～ 去 大官 屯 那取 的 。 虽然 连 卖家 的 面 都 没 见到 ～ 但是 还是 谢谢 卖家 送 我 这么 可爱 的 东西 ～ 太 喜欢 了 ～ 这 哪 舍得吃 啊 。	__label__9_1

餐前 ～ 装 冷菜 的 碟子 是 仿 青瓷 的 碟子 ～ 高脚 ～ 可惜 是 塑料 的 ～ 近 看 缺 了 质感 ～ 没有 瓷器 那种 通透 ～ 远看 挺不错 。 盛菜 的 器皿 有些 也 挺 有 特色 的 ～ 包厢 布置 也 花 了 功夫 ～ 走 古风 挂 字画 。 墙纸 是 青花瓷 的 ～ 对面 墙壁 也 贴 了 很多 瓷器 ～ 有 几个 还是 玲珑 瓷 ～ 透光 看 比较 好看 。 比较 喜欢 他们 家 的 日式 茶壶 ～ 铁壶 ～ 很沉 但 漂亮 ～ 据说 冲泡 能 补铁 ？ 菜品 味道 一般 ～ 花样 是 挺 多 的 ～ 杨枝 甘露 换个 器皿 就 能 换来 很大 的 拍照 量 ～ 是不是 桌桌 都 有 ？ 喜欢 那个 有 小 把 的 酱料 器皿 ～ 拿 着 很 可爱	__label__9_-2

非常 喜欢 去 新开 的 有 特色 的 店 品尝 ～ 有幸 被 大众 点评 网同城 聚会 抽到 黄金 替补 免费 品鉴 的 机会 ～ 我 感到 十分 幸运 ！ 红 不让 台湾 特色 餐厅 位于 李沧 万达 金街 1 号 入口 扶梯 旁 ～ 交通 便利 ～ 停车 方便 。 店面 小资 ～ 适合 举行 小型 派对 。 走近 店门 ～ 服务员 立刻 开门 迎接 并且 热情 的 打招呼 。 进入 店面 ～ 环顾四周 ～ 鹅黄 的 灯光 下 透露 出 温馨 慵懒 的 气氛 ～ 店家 匠心 独居 ～ 屋内 的 摆设 典雅 质朴 。 

哎 ～ 想当年 来 佘山 的 时候 ～ 啥 都 没有 ～ 三品 香算 镇上 最大 看起来 最 像样 的 饭店 了 。 菜品 多 ～ 有点 太 多 ～ 感觉 啥 都 有 ～ 杂都 不足以 形容 。 随便 点些 ～ 居然 口味 什么 的 都 好 还 可以 ～ 价钱 自然 是 便宜 当 震惊 。 元宝 虾 和 椒盐 九肚鱼 都 不错 吃 。 不过 近来 几次 么 ～ 味道 明显 没 以前 好 了 。 冷餐 里面 一个 凉拌 海带丝 还 可以 ～ 酸酸甜甜 的 。 镇上 也 有 了 些 别的 大点 的 饭店 ～ 所以 不是 每次 必来 了 。 对 了 ～ 这家 的 生意 一如既往 的 超级 好 ～ 不 定位 基本 吃 不到 。 不过 佘山 这边 的 人 吃晚饭 很早 的 ～ 所以 稍微 晚点 去 就 很 空 了 。	__label__9_-2


File generated! 


## 结论

这里的f1和recall没有参考价值。

但是precision比elmo based深度模型每种label至少低了2%，最多低了8%以上。

一次预测所有label的precision只有70%左右。

划分为80个二分类，整体的precision也没有显著的提升。

### 量化

In [ ]:
def quanti(input_file, model):
    # with the previously trained `model` object, call :
    model.quantize(input=input_file, retrain=True)
    return model

In [17]:
# !wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz
# !head cooking.stackexchange.txt
# ! wc cooking.stackexchange.txt
# ! head -n 12404 cooking.stackexchange.txt > cooking.train
# ! tail -n 3000 cooking.stackexchange.txt > cooking.valid